In [5]:
import os
import random
##Escoge 100 archivos al azar
folder_path = "reviews"  
all_files = os.listdir(folder_path)
random_files = random.sample(all_files, 100)
print(random_files)

['35_3.txt', '74_3.txt', '20_1.txt', '51_1.txt', '96_1.txt', '46_4.txt', '90_4.txt', '75_1.txt', '79_4.txt', '9_1.txt', '11_3.txt', '71_1.txt', '16_3.txt', '53_3.txt', '72_4.txt', '0_3.txt', '92_3.txt', '25_1.txt', '37_3.txt', '83_3.txt', '64_1.txt', '22_1.txt', '48_4.txt', '1_1.txt', '34_1.txt', '94_1.txt', '4_4.txt', '87_4.txt', '2_1.txt', '70_2.txt', '95_3.txt', '67_2.txt', '91_1.txt', '99_1.txt', '69_4.txt', '42_3.txt', '36_4.txt', '6_1.txt', '52_1.txt', '88_2.txt', '23_3.txt', '30_1.txt', '29_4.txt', '10_2.txt', '31_1.txt', '56_3.txt', '65_4.txt', '18_3.txt', '40_3.txt', '86_4.txt', '89_2.txt', '78_4.txt', '61_3.txt', '77_4.txt', '57_4.txt', '27_1.txt', '76_3.txt', '93_1.txt', '28_2.txt', '5_3.txt', '98_1.txt', '44_2.txt', '41_1.txt', '63_1.txt', '17_3.txt', '97_1.txt', '85_2.txt', '12_1.txt', '32_3.txt', '55_1.txt', '73_1.txt', '84_3.txt', '3_4.txt', '38_2.txt', '19_4.txt', '21_4.txt', '47_2.txt', '7_3.txt', '60_4.txt', '81_1.txt', '66_4.txt', '68_2.txt', '54_1.txt', '82_1.txt', 

In [6]:
##Creacion de diccionario de peliculas
movies_dict = {}
for file in random_files:
    new_file = file.split('_')
    new_new_file = new_file[1].split('.')
    key = new_new_file[0]
    if key in movies_dict:
        movies_dict[key].append(file)
    else:
        movies_dict[key] = [file]
print(movies_dict)


{'3': ['35_3.txt', '74_3.txt', '11_3.txt', '16_3.txt', '53_3.txt', '0_3.txt', '92_3.txt', '37_3.txt', '83_3.txt', '95_3.txt', '42_3.txt', '23_3.txt', '56_3.txt', '18_3.txt', '40_3.txt', '61_3.txt', '76_3.txt', '5_3.txt', '17_3.txt', '32_3.txt', '84_3.txt', '7_3.txt', '80_3.txt', '26_3.txt', '33_3.txt', '58_3.txt', '59_3.txt'], '1': ['20_1.txt', '51_1.txt', '96_1.txt', '75_1.txt', '9_1.txt', '71_1.txt', '25_1.txt', '64_1.txt', '22_1.txt', '1_1.txt', '34_1.txt', '94_1.txt', '2_1.txt', '91_1.txt', '99_1.txt', '6_1.txt', '52_1.txt', '30_1.txt', '31_1.txt', '27_1.txt', '93_1.txt', '98_1.txt', '41_1.txt', '63_1.txt', '97_1.txt', '12_1.txt', '55_1.txt', '73_1.txt', '81_1.txt', '54_1.txt', '82_1.txt', '15_1.txt', '24_1.txt'], '4': ['46_4.txt', '90_4.txt', '79_4.txt', '72_4.txt', '48_4.txt', '4_4.txt', '87_4.txt', '69_4.txt', '36_4.txt', '29_4.txt', '65_4.txt', '86_4.txt', '78_4.txt', '77_4.txt', '57_4.txt', '3_4.txt', '19_4.txt', '21_4.txt', '60_4.txt', '66_4.txt', '8_4.txt', '50_4.txt', '49_4

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
directory = 'reviews/'

for movie in movies_dict:
    moviechosen = movies_dict[movie]
    
    lemmatized_directory = 'lemmatized_reviews/'
    os.makedirs(lemmatized_directory, exist_ok=True)
    
    for file in moviechosen:
        path = os.path.join(directory, file)
        
        with open(path, 'r') as archivo:
            text = archivo.read()
            text=text.lower()
            text=text.replace("< br / >","")
            text=text.replace("<br />","")
            text = text.translate(str.maketrans('', '', string.punctuation))
            tokens = nltk.word_tokenize(text)
            stop_words = set(stopwords.words('english'))
            filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
            lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
            lemmatized_text = ' '.join(lemmatized_tokens)
            
            # Create a new file with lemmatized content
            lemmatized_path = os.path.join(lemmatized_directory, file)
            with open(lemmatized_path, "w") as output_file:
                output_file.write(lemmatized_text)

print("Lemmatized files have been created in the 'lemmatized_reviews' directory.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Lemmatized files have been created in the 'lemmatized_reviews' directory.


In [8]:
import os

input_folder = 'lemmatized_reviews/'
output_folder = 'reviews_by_movie/'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

file_contents = {}
for filename in os.listdir(input_folder):
    file_number = filename.split("_")[1].split('.')[0]

    with open(os.path.join(input_folder, filename), 'r') as input_file:
        file_content = input_file.read()

    if file_number not in file_contents:
        file_contents[file_number] = []
    file_contents[file_number].append(file_content)

for file_number, contents in file_contents.items():
    output_file_path = os.path.join(output_folder, f"{file_number}.txt")
    with open(output_file_path, 'w') as output_file:
        output_file.write("\n".join(contents))

print("Files have been grouped and written to separate output files.")

Files have been grouped and written to separate output files.


In [15]:
from collections import Counter
import string
import os

def get_term_frequency_in_file(source_file_path):
    wordcount = {}
    with open(source_file_path) as f:
        translator = str.maketrans('', '', string.punctuation)  # Create a translation table
        for line in f:
            line = line.lower().translate(translator)  # Use the translation table to remove punctuation
            this_wordcount = Counter(line.split())
            wordcount = add_merge_two_dict(wordcount, this_wordcount)
    return wordcount

def add_merge_two_dict(x, y):
    return { k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y) }

input_folder = 'reviews_by_movie/'
output_folder = 'word_count_by_movie/'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    file_number = filename.split('.')[0]

    wordCount = get_term_frequency_in_file(input_folder + filename)

    output_file_path = os.path.join(output_folder, f"{file_number}_wordcount.txt")
    with open(output_file_path, 'w') as output_file:
        output_file.write(str(wordCount))

print("Los vectores fueron escritos correctamente")